First let's generate a few random points on a 2D plane.

We have to import a few things, namely:
- numpy for generating random numbers
- matplotlib.pyplot for plotting
- cv2 for image processing
- io for reading and writing files into memory
- scipy.spatial for calculating the Voronoi diagram

In [ ]:
import cv2
import io
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d

Let's setup our figures. We're gonna need 3 subplots:
- the base image
- openCV processing output
- Voronoi diagram.

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5)) # (5+5+5)*5

Generate black donts on a white background to use as our base image

In [ ]:
def generate_image(ax: plt.Axes, img: io.BytesIO,n_points: int = 10):
    # Generate random points
    points = np.random.rand(n_points, 2)

    # Turn off axis and labels
    ax.axis("off")
    ax.xticks([])
    ax.yticks([])

    # Plot the points as black dots
    ax.scatter(points[:, 0], points[:, 1], s=1, c="k")

    # Save the figure to the buffer
    ax.savefig(img, bbox_inches="tight", pad_inches=0, format="png")
    ax.show()